In [28]:
import json
from typing import Dict, Tuple
from google import genai 
from environment import GEMINI_API_KEY
import re
from IPython.display import display, HTML

In [29]:
def get_divide_prompt(input_str, output_str):
    return f"""
    **Prompt: Divide Reasoning into Sequential Steps**

    **Objective:** To process a given reasoning text and divide it into a sequence of distinct steps, capturing the flow of thought.

    **Your Task:**
    Read the `REASONING_TEXT` provided below, which explains the thinking process for solving the `PROBLEM_DESCRIPTION`. Your goal is to break this reasoning text down into sequential, logical steps. For each step you identify:
    1.  Assign a unique `step` number, starting from 1.
    2.  Write a brief `summary` describing the main point or action of that step.
    3.  Include the exact segment of the original `text` that constitutes this step.

    **Inputs You Will Receive:**
    * `PROBLEM_DESCRIPTION`: The text of the problem.
    * `REASONING_TEXT`: The text containing the step-by-step reasoning.

    **Required Output Format:**
    You must output ***only*** a single JSON object. Do not add any explanations before or after the JSON. The JSON object should contain a list called `steps`. Each item in the list represents one step and must strictly follow this structure:

    ```json
    {{
        "steps": [
            {{
                "step": 1,
                "summary": "Example summary for step 1",
                "text": "Example text segment for step 1"
            }},
            {{
                "step": 2,
                "summary": "Example summary for step 2",
                "text": "Example text segment for step 2"
            }}
            // ... more step objects covering the entire reasoning text
        ]
    }}
    ```
    Instructions Summary:

    Divide the entire REASONING_TEXT into consecutive, meaningful steps based on the flow of reasoning. For each step, provide the step number, a concise summary, and the corresponding text segment. Ensure the output is a single, valid JSON object matching the specified format exactly.

    --- START OF INPUT ---
    PROBLEM_DESCRIPTION:
    {input_str}

    REASONING_TEXT:
    {output_str}
    --- END OF INPUT ---
    """
    
def get_tree_prompt(parsed_steps):
    return f"""
    **Prompt: Analyze Reasoning, Cluster Steps, and Determine Logical Flow Tree (Two-Part Output)**

    **Objective:** To process a sequence of reasoning steps, identify the underlying core logical functions (including distinct approaches), group the steps accordingly, determine the tree structure representing the exploration of these approaches and the successful path, and output these results as distinct parts within a single JSON object.

    **Your Task:**
    You will receive a JSON object containing a list of sequential reasoning steps (`steps`), where each step includes a `step` number, `summary`, and `text`. Your comprehensive task is to perform the following analysis in sequence:

    1.  **Identify & Define Logical Functions/Approaches:**
        * Analyze all input steps to identify the distinct logical functions or phases performed. Pay attention to explicit mentions of different **Approaches**.
        * For each distinct function or phase within an approach (e.g., Stating Approach 1, Executing Approach 1, Evaluating Approach 1), create a concise descriptive label and assign a unique code (`A1`, `A2`, `B1`, `C1`, etc.). Use your judgment to decide the granularity needed to represent the reasoning accurately.
        * Treat routine verification or re-calculation steps as part of the function they verify, unless they trigger significant confusion or method changes that constitute their own phase (as per previous refinement).

    2.  **Cluster Steps by Function:**
        * Classify each step from the input `steps` list according to the logical functions/codes you defined in Task 1.
        * Group the `step` numbers based on the function code they are assigned to, ensuring every input step is assigned to a cluster.

    3.  **Determine Logical Flow (Tree Structure with Branches):**
        * Based on the clustering results and the reasoning narrative, identify the overall tree structure, including alternative approaches.
        * Determine the "starting node code" (like `A1`, `B1`, `C1`...) for each distinct top-level approach attempted.
        * Trace the sequence of function codes *within* each distinct approach branch.
        * Identify which approach branch leads to the final successful conclusion.
        * Represent this structure as a tree within the `logical_flow` object:
            * Include a special key `"root"` whose value is a list of the starting node codes for **all** distinct top-level approaches identified.
            * For each node code defined (`A1`, `A2`, etc.), create a key. Its value should be a list containing the node code(s) that *directly follow it within its specific approach branch*.
            * Nodes that represent the end of a failed or abandoned approach branch should have an empty list `[]` as their value.
            * The **final node** of the **successful** approach branch should have a list containing only the string `"solution"` (i.e., `["solution"]`) as its value.

    **Input:**
    * `STEP_1_OUTPUT_JSON`: A JSON object containing a list called `steps`. Each element has `step`, `summary`, and `text`. (Example structure as before).

    **Required Output Format:**
    Produce ***only*** a single JSON object containing exactly two top-level keys: `clustering_results` and `logical_flow`. Adhere strictly to the following structure:

    ```json
    {{
        "clustering_results": {{
            // Part 1 Output: Clustering Information
            // Keys: Uppercase letter codes (A1, A2, B1...) identified from Task 1.
            // Values: Objects containing:
            //    "description": (string) The concise descriptive label for the function/phase (from Task 1).
            //    "steps": (list of integers) The step numbers performing this function (from Task 2).
            // Example (content depends on analysis):
            // "A1": {{ "description": "Approach 1: State Method...", "steps": [1] }},
            // "A2": {{ "description": "Approach 1: Execute Tests...", "steps": [2, 3, 4, 5, 6] }},
            // "A3": {{ "description": "Approach 1: Evaluate Method...", "steps": [7] }},
            // ... etc. for B1-B3, C1-C4, D1-D4 ...
        }},
        "logical_flow": {{
            // Part 2 Output: Tree Flow Information
            // Matches the structure requested by the user.
            // Includes a "root" key pointing to starting nodes of each approach.
            // Other keys are node codes, values are lists of direct successors within that approach branch.
            // Failed branches end with []. Successful branch ends pointing to "solution".
            // Example (Based on user's desired structure and A1..D4 nodes):
            "root": ["A1", "B1", "C1", "D1"],
            "A1": ["A2"],
            "A2": ["A3"],
            "A3": [], // End of failed Approach 1
            "B1": ["B2"],
            "B2": ["B3"],
            "B3": [], // End of failed Approach 2
            "C1": ["C2"],
            "C2": ["C3"],
            "C3": ["C4"],
            "C4": [], // End of failed Approach 3
            "D1": ["D2"],
            "D2": ["D3"],
            "D3": ["D4"],
            "D4": ["solution"] // End of successful Approach 4
        }}
    }}
    Instructions Summary:

    Perform the three tasks (Identify/Define Functions, Cluster Steps, Determine Flow Tree) based on the input steps.
    Populate the clustering_results section. Ensure all input steps are clustered.
    Populate the logical_flow section using the exact tree structure specified, including the "root" key and ["solution"] endpoint for the successful path. Failed paths should end with [].
    Ensure the codes (A1, B1, etc.) are consistent across both output sections.
    Output only the single JSON object matching the specified two-part structure exactly. No additional text.
    --- START OF INPUT ---
    STEP_1_OUTPUT_JSON:
    {parsed_steps}
    --- END OF INPUT ---
    """

def generate_node_positions(logical_flow: Dict) -> Dict[str, Tuple[float, float]]:
    """Generate x, y coordinates for each node based on the logical flow structure."""
    positions = {}
    
    # Count approaches to distribute them
    root_children = logical_flow.get("root", [])
    num_approaches = len(root_children)
    
    # Set root position
    positions["root"] = (400, 50)
    
    # Special case handling: D approach needs more space
    approach_widths = {
        "A": 200,
        "B": 400,
        "C": 600,
        "D": 700
    }
    
    # Process each approach
    for approach_id in root_children:
        approach_letter = approach_id[0]
        x_pos = approach_widths.get(approach_letter, 0)
        
        # Position first node in approach
        positions[approach_id] = (x_pos, 120)
        
        # Track current y position as we go down the chain
        current_y = 120
        current_node = approach_id
        
        # Follow the chain in this approach
        while current_node in logical_flow and logical_flow[current_node]:
            next_node = logical_flow[current_node][0]  # Get first (should be only) child
            current_y += 80  # Increment y position for next level
            positions[next_node] = (x_pos, current_y)
            current_node = next_node
    
    # Add solution node if in the flow
    if "solution" in logical_flow.get("D4", []):
        positions["solution"] = (700, 440)
    
    return positions

def create_node_descriptions(clustering_results: Dict) -> Dict[str, str]:
    """Extract short descriptions for each node."""
    descriptions = {}
    for node_id, info in clustering_results.items():
        descriptions[node_id] = info.get("description", "")
    return descriptions

def generate_trajectory_path(logical_flow: Dict, positions: Dict[str, Tuple[float, float]]) -> str:
    """Generate SVG path for the trajectory through the nodes."""
    # Start with root
    path = f"M {positions['root'][0]} {positions['root'][1]} "
    
    # Follow each approach in sequence
    approaches = logical_flow.get("root", [])
    
    for approach_id in approaches:
        # Add path to the first node of this approach
        x, y = positions[approach_id]
        # Add a curve to make it look nice
        cx1, cy1 = positions['root'][0] - 150, positions['root'][1] + 20
        cx2, cy2 = x - 50, y - 20
        path += f"\n           C {cx1} {cy1}, {cx2} {cy2}, {x} {y}"
        
        # Follow the chain in this approach
        current_node = approach_id
        
        while current_node in logical_flow and logical_flow[current_node]:
            next_node = logical_flow[current_node][0]
            nx, ny = positions[next_node]
            
            # Add a nice curve
            cx1, cy1 = x + 20, y + 10
            cx2, cy2 = nx, ny - 40
            path += f"\n           C {cx1} {cy1}, {cx2} {cy2}, {nx} {ny}"
            
            current_node = next_node
            x, y = nx, ny
        
        # If not the last approach, add a curve back to the root of the next approach
        if approach_id != approaches[-1]:
            next_approach = approaches[approaches.index(approach_id) + 1]
            next_x, next_y = positions[next_approach]
            
            # Curve back up and then to the next approach
            cx1, cy1 = x, y + 30
            cx2, cy2 = next_x - 100, next_y - 30
            path += f"\n           C {cx1} {cy1}, {cx2} {cy2}, {next_x} {next_y}"
            
            x, y = next_x, next_y
    
    return path

def generate_svg(data: Dict) -> str:
    """Generate SVG visualization from the parsed data."""
    clustering_results = data.get("clustering_results", {})
    logical_flow = data.get("logical_flow", {})
    
    # Generate positions for each node
    positions = generate_node_positions(logical_flow)
    
    # Get descriptions for nodes
    descriptions = create_node_descriptions(clustering_results)
    
    # Define node colors for each approach
    node_colors = {
        'root': '#3498db',
        'A': '#2ecc71',
        'B': '#9b59b6',
        'C': '#f39c12',
        'D': '#e74c3c',
        'solution': '#27ae60'
    }
    
    # Start building SVG
    svg = f"""<svg viewBox="0 0 800 600" xmlns="http://www.w3.org/2000/svg">
  <!-- Styles -->
  <defs>
    <marker id="arrowhead" markerWidth="10" markerHeight="7" refX="9" refY="3.5" orient="auto">
      <polygon points="0 0, 10 3.5, 0 7" fill="#555"/>
    </marker>
    <marker id="trajectory-arrowhead" markerWidth="10" markerHeight="7" refX="9" refY="3.5" orient="auto">
      <polygon points="0 0, 10 3.5, 0 7" fill="#e74c3c"/>
    </marker>
  </defs>
  
  <!-- Tree Structure -->"""
    
    # Add connections between nodes
    for parent, children in logical_flow.items():
        # Skip if parent not in positions (like 'solution')
        if parent not in positions:
            continue
            
        px, py = positions[parent]
        
        for child in children:
            # Skip if child not in positions
            if child not in positions:
                continue
                
            cx, cy = positions[child]
            svg += f"""
  <line x1="{px}" y1="{py}" x2="{cx}" y2="{cy}" stroke="#555" stroke-width="2" marker-end="url(#arrowhead)"/>"""
    
    # Add trajectory path
    trajectory_path = generate_trajectory_path(logical_flow, positions)
    svg += f"""
  
  <!-- Trajectory curve with arrow -->
  <path d="{trajectory_path}"
        fill="none" stroke="#e74c3c" stroke-width="3" stroke-dasharray="5,3" marker-end="url(#trajectory-arrowhead)"/>
  
  <!-- Nodes -->"""
    
    # Add root node
    svg += f"""
  <!-- Root -->
  <circle cx="{positions['root'][0]}" cy="{positions['root'][1]}" r="20" fill="{node_colors['root']}"/>
  <text x="{positions['root'][0]}" cy="{positions['root'][1] + 5}" text-anchor="middle" fill="white" font-weight="bold">Root</text>
  """
    
    # Add all other nodes
    for node_id, (x, y) in positions.items():
        if node_id == 'root' or node_id == 'solution':
            continue
            
        approach_letter = node_id[0]
        color = node_colors.get(approach_letter, '#777')
        
        svg += f"""
  <circle cx="{x}" cy="{y}" r="20" fill="{color}"/>
  <text x="{x}" cy="{y + 5}" text-anchor="middle" fill="white" font-weight="bold">{node_id}</text>
  """
    
    # Add solution node if present
    if 'solution' in positions:
        x, y = positions['solution']
        svg += f"""
  <!-- Solution node -->
  <circle cx="{x}" cy="{y}" r="20" fill="{node_colors['solution']}"/>
  <text x="{x}" cy="{y + 5}" text-anchor="middle" fill="white" font-weight="bold">Sol</text>
  """
    
    # Add legend
    svg += """
  <!-- Legend -->
  <rect x="50" y="500" width="250" height="90" rx="10" ry="10" fill="#f8f9fa" stroke="#ddd"/>
  <text x="60" y="520" font-weight="bold">Legend:</text>
  <circle cx="70" cy="540" r="7" fill="#3498db"/>
  <text x="85" y="545">Root node</text>
  <line x1="170" y1="540" x2="210" y2="540" stroke="#555" stroke-width="2" marker-end="url(#arrowhead)"/>
  <text x="220" y="545">Logical flow</text>
  <line x1="60" y1="565" x2="100" y2="565" stroke="#e74c3c" stroke-width="3" stroke-dasharray="5,3" marker-end="url(#trajectory-arrowhead)"/>
  <text x="110" y="570">Solution trajectory</text>
  """
    
    # Add descriptions
    for node_id, description in descriptions.items():
        if node_id not in positions:
            continue
            
        x, y = positions[node_id]
        # Add offset for text positioning
        svg += f"""
  <text x="{x + 50}" y="{y}" text-anchor="start" font-size="10">{description}</text>"""
    
    # Add title
    svg += """
  
  <text x="400" y="20" text-anchor="middle" font-size="14" font-weight="bold">Problem-Solving Approach Flow and Trajectory</text>
</svg>"""
    
    return svg

def parse_json(json_prompt):
    """Parse JSON content from a prompt string.
    
    Args:
        json_prompt: A string containing JSON data between ```json and ``` markers
        
    Returns:
        The parsed JSON data as a Python dictionary
    """
    
    # Find content between ```json and ``` markers
    json_match = re.search(r'```json\s*(.*?)\s*```', json_prompt, re.DOTALL)
    
    if not json_match:
        return {}
    
    json_content = json_match.group(1)
    
    # Parse the JSON content
    data = json.loads(json_content)
    
    return data
    

In [30]:
client = genai.Client(api_key=GEMINI_API_KEY)

def call_gemini(prompt, model_name = "gemini-2.5-pro-preview-03-25"):
    response = client.models.generate_content(
        model=model_name,
        contents=[prompt],
    )
    return response.candidates[0].content.parts[0].text

In [31]:
input_str = "Find the number of positive integer solutions to the equation x^2 - y! = 0, where x and y are positive integers and y! denotes the factorial of y."
output_str = "Approach 1: Brute Force Enumeration\nLet’s try solving by testing small values of y and checking if y! is a perfect square, since x^2 = y! implies x = √(y!) must be an integer.\n- For y = 1, y! = 1! = 1 = 1^2, so x = 1. Solution: (x, y) = (1, 1).\n- For y = 2, y! = 2! = 2. Since √2 ≈ 1.414, not an integer, no solution.\n- For y = 3, y! = 3! = 6. Since √6 ≈ 2.449, not an integer, no solution.\n- For y = 4, y! = 4! = 24. Since √24 ≈ 4.899, not an integer, no solution.\n- For y = 5, y! = 5! = 120. Since √120 ≈ 10.954, not an integer, no solution.\nFailure: This approach is inefficient for large y, as factorials grow rapidly, and computing square roots manually is tedious. It confirms (1, 1), but we need a systematic way to find all solutions or prove there are no others.\n\nApproach 2: Prime Factorization Analysis\nSince x^2 = y!, y! must be a perfect square. A number is a perfect square if, in its prime factorization, all exponents are even. Let’s compute the prime factorization of y! and check if all exponents are even.\nFor y = 4:\n- 4! = 24 = 2^3 · 3^1.\n- Exponents: 3 (for 2) and 1 (for 3) are odd, so 24 is not a perfect square.\nFor y = 5:\n- 5! = 120 = 2^3 · 3^1 · 5^1.\n- Exponents: 3, 1, 1 are odd, so 120 is not a perfect square.\nTry a larger y, say y = 6:\n- 6! = 720 = 2^4 · 3^2 · 5^1.\n- Exponents: 4 (even), 2 (even), 1 (odd), so 720 is not a perfect square.\nFailure: For y! = 1 · 2 · … · y, the exponent of a prime p in y! is ∑_{k≥1} ⌊y / p^k⌋, which is typically odd for many primes unless carefully constructed. This approach is complex to generalize, as factorials involve products of all numbers up to y, making it hard to ensure all exponents are even.\n\nApproach 3: Modular Arithmetic\nTo constrain solutions, test the equation modulo a small number. Try modulo 4, as squares have specific residues.\nSquares modulo 4:\n- 0^2 ≡ 0, 1^2 ≡ 1, 2^2 ≡ 0, 3^2 ≡ 1 (mod 4).\n- Thus, x^2 ≡ 0 or 1 (mod 4).\nNow compute y! (mod 4):\n- y = 1: 1! = 1 ≡ 1 (mod 4). Possible, as x^2 ≡ 1.\n- y = 2: 2! = 2 ≡ 2 (mod 4). Not possible, as x^2 ≢ 2.\n- y = 3: 3! = 6 ≡ 2 (mod 4). Not possible.\n- y ≥ 4: For y ≥ 4, y! includes at least two factors of 2 (e.g., 2 and 4), so y! ≡ 0 (mod 4). Possible, as x^2 ≡ 0.\nThis suggests y = 2, 3 are impossible, but y = 1 or y ≥ 4 may work. Let’s test y ≥ 4:\n- y = 4: 4! = 24, not a square (as before).\n- y = 5: 5! = 120, not a square.\nFailure: Modular arithmetic filters out some y, but for y ≥ 4, y! ≡ 0 (mod 4), which aligns with x^2 ≡ 0, yet most y! are not perfect squares. This approach doesn’t efficiently identify when y! is a square.\n\nApproach 4: Combinatorial Insight and Systematic Testing (Successful)\nRecognize that y! grows factorially, and we need y! = x^2. Since y = 1 worked (1! = 1 = 1^2), let’s hypothesize that larger y may produce factorials that are squares due to the product structure. Instead of computing factorials fully, consider the pattern of factorials and their likelihood of being squares.\nNotice that factorials include many primes and composite numbers, making odd exponents likely. Let’s try a few more y, but with a focus on small y, as large factorials grow too fast to be squares.\nTry y = 6:\n- 6! = 720. Check if 720 is a square: √720 ≈ 26.833, not an integer.\nTry y = 7:\n- 7! = 7 · 6! = 7 · 720 = 5040.\n- Check: √5040 ≈ 70.993, not an integer.\nNow try y = 8:\n- 8! = 8 · 7! = 8 · 5040 = 40320.\n- Check: √40320 ≈ 200.798, not an integer.\nThis is still inefficient, so let’s refine our hypothesis. Factorials are products of consecutive integers, and for y! to be a square, the number of factors must balance to produce even exponents. For small y, let’s try one more that seems promising by computing factorials up to a point where we suspect no further solutions.\nTry y = 9:\n- 9! = 362880.\n- Check: √362880 ≈ 602.395, not an integer.\nAt this point, observe that y! for y ≥ 2 produces large numbers that are rarely squares due to the presence of primes with odd exponents (e.g., the largest prime in y!). Let’s formalize: for y ≥ 2, y! includes at least one prime with an odd exponent unless y is constructed specifically (e.g., y = 1).\nTo confirm, consider the prime factorization of y!. The exponent of a prime p ≤ y is: e_p = ∑_{k≥1} ⌊y / p^k⌋. For large y, many primes appear with exponent 1 (if p is close to y), making odd exponents common. For y = 1, 1! = 1, a perfect square. For y = 2, 2! = 2, not a square. For higher y, the mix of primes suggests y! is unlikely to have all even exponents.\nConclusion: After testing and analyzing, the only y where y! is a perfect square is y = 1, as higher factorials introduce primes with odd exponents. Thus, the only solution is: y = 1, y! = 1, x = 1.\nFinal Answer: The only positive integer solution is (x, y) = (1, 1)."

In [32]:
divide_prompt = get_divide_prompt(input_str, output_str)
parsed_steps = call_gemini(divide_prompt)
print(parsed_steps)


```json
{
    "steps": [
        {
            "step": 1,
            "summary": "Introduce Approach 1: Brute Force Enumeration. Rewrite equation as x = sqrt(y!) and plan to test small values of y.",
            "text": "Approach 1: Brute Force Enumeration\nLet’s try solving by testing small values of y and checking if y! is a perfect square, since x^2 = y! implies x = √(y!) must be an integer."
        },
        {
            "step": 2,
            "summary": "Test y=1: Find y! = 1, which is 1^2, yielding the solution (x, y) = (1, 1).",
            "text": "- For y = 1, y! = 1! = 1 = 1^2, so x = 1. Solution: (x, y) = (1, 1)."
        },
        {
            "step": 3,
            "summary": "Test y=2: Find y! = 2, which is not a perfect square.",
            "text": "- For y = 2, y! = 2! = 2. Since √2 ≈ 1.414, not an integer, no solution."
        },
        {
            "step": 4,
            "summary": "Test y=3: Find y! = 6, which is not a perfect square.",
            "text": "

In [33]:
tree_prompt = get_tree_prompt(parsed_steps)
output_json = call_gemini(tree_prompt)
print(output_json)

```json
{
    "clustering_results": {
        "A1": {
            "description": "Approach 1: State Brute Force Enumeration & Plan",
            "steps": [
                1
            ]
        },
        "A2": {
            "description": "Approach 1: Execute Brute Force Tests (y=1 to 5)",
            "steps": [
                2,
                3,
                4,
                5,
                6
            ]
        },
        "A3": {
            "description": "Approach 1: Evaluate Brute Force (Inefficient)",
            "steps": [
                7
            ]
        },
        "B1": {
            "description": "Approach 2: State Prime Factorization Analysis",
            "steps": [
                8
            ]
        },
        "B2": {
            "description": "Approach 2: Define Condition & Execute Analysis (y=4, 5, 6)",
            "steps": [
                9,
                10,
                11,
                12
            ]
        },
        "B3": 

In [34]:
json_data = parse_json(output_json)
svg = generate_svg(json_data)
# Try to display the SVG in the notebook
display(HTML(svg))
